# Nathaniel Mallick
## Project 5 
## Discrete Network Events

### Project partners - Justin Cacal 


## IMPORTS


In [1]:
from vpython import *
from queue import PriorityQueue

<IPython.core.display.Javascript object>


## NET/MODULE/EVENT CLASS DEFINTIONS 
### All classes used in this project are defined in this block plus helper functions to run a clock signal 


In [2]:
##############################################################    
                    ## HELPER FUCNTION ## 
def clock(val, obj): 
    if val == 0: 
        if obj.value == "X":
            obj.value = "0"
        elif obj.value == "0":
            obj.value = "1"
        elif obj.value == "1":
            obj.value = "0"
##############################################################    

##############################################################    
######################### "NET" CLASS ########################
class net: 
    __signal_types = ["0", "1", "X"]
    
    def __init__(self, id, val="X"): 
        self.driver = None
        self.receivers = []
        self.id = id
        self.prev_value = val
        self.value = val
        self.data = []
            
    def update(self): 
        if self.value == "X": 
            self.data.append(0.5)
        else: 
            self.data.append(int(self.value))
##############################################################    
##############################################################    
     
##############################################################    
######################## "GATE" CLASS ########################
class module: 
    __gate_types = ["NAND", "NOR"]
    __signal_types = ["0", "1", "X"]

    def __init__(self, input, output, func, id, dt=1): 
        ## Time for delay ##
        self.time = 0
        self.dt = dt 
        
        self.delay = 5 ## delay in nanoseconds 
        self.q = [] ## "FIFO" queue but as a list 
        
        ## HOLD THE STATE OF THE MODULE ## 
        self.valid = True
        
        ## VALIDATE MODULE INPUTS/FUNCTION ## (not-important)
        if not( func.upper() in module.__gate_types ): 
            print("Gate function not found.")
            self.valid = False
        
        if len(input) != 2:
            print("Invalid input net size.")
            self.valid = False
            
        if not( input[0].value in module.__signal_types ) or not( input[1].value in module.__signal_types):
            print("Invalid input type.")
            self.valid = False
        
        ## IMPORTANT ##
        self.id = id
        self.inputs = input ## must be an array with 2 net elements( 2 inputs ) 
        self.output = output ## net 
        self.func = func
        self.event()

    def event(self):        
        e = {} ## OBJECT TO HOLD THE EVENT
        
        ## OUTPUT BASED ON FUNCTION ## 
        if self.inputs[0].value == "X" or self.inputs[1].value == "X":
            e["output"] = "X"
        elif self.func == "NAND": 
            val = not(int(self.inputs[0].value) and int(self.inputs[1].value))
            e["output"] = str(int(val))
        elif self.func == "NOR":
            val = not(int(self.inputs[0].value) or int(self.inputs[1].value))
            e["output"] = str(int(val))
        
        ## DELAY EVENT ## 
        e["trigger"] = self.time + self.delay 
        self.q.append(e)
    
    def update(self):  
        if len(self.q) != 0:     
            ## CHECK IF THE DELAY TIME HAS BEEN REACHED ## 
            for i,obj in enumerate(self.q): 
                if self.time >= obj["trigger"]:
                    self.output.value = obj["output"]
                    self.q.pop(i) ## REMOVE THE EVENT IF IT HAPPENED ##
                    
        self.time += self.dt

##############################################################
##############################################################


##############################################################    
######################## "EVENT" CLASS #######################
class event: 
    def __init__(self, nets, dt=1): 
        self.nets = nets ## should be an array/list 
        self.time = 0
        self.data = [] ## time step value s
        self.dt = dt
        
    def listen(self):
        for i in self.nets: 
            if i.value != i.prev_value: 
                ## que event
                for j in i.receivers:
                    j.event()
            i.prev_value = i.value
            i.update()
            
        self.data.append(self.time)
        self.time += self.dt 
##############################################################
##############################################################            


# TESTING EACH CLASS
## Gates are tested by forcing all combination of inputs. The net class has a constructor that initiates the value of the drivers. 



## NAND TESTS


In [6]:
scene = canvas()
## TIME ARRAY FOR PLOTTING ##
t_arr = []

## NETS AND MODULES ## 
## DEFAULT VALUE FOR NETS IS "X"
net1 = net(val="1", id='1') ## driver
net2 = net(val="1", id='2') ## driver
net3 = net(id='3') ## reciever 

nand_gate = module( input=[net1, net2],
                    output=net3,
                    func="NAND",
                    id='1'
                    )

## UPDATE THE MODULES ## 
## DELAY = 5NS 
## X (undefined) = 0.5 
## A CHANGE SHOULD HAPPEN AFTER THE 5 TIME CYCLE 
for i in range(10):
    nand_gate.update()
    net1.update()
    net2.update()
    net3.update()
    t_arr.append(nand_gate.time)

    
## GRAPH INPUT NET 1
graph(ymin=-0.5, ymax=1.5)
n1 = gcurve(color=color.blue, label="NET 1 (driver)")
for i in range(len(t_arr)):
    n1.plot(t_arr[i], net1.data[i])

## GRAPH INPUT NET 2
graph(ymin=-0.5, ymax=1.5)
n2 = gcurve(color=color.black, label="NET 2 (driver)")
for i in range(len(t_arr)):
    n2.plot(t_arr[i], net2.data[i])
    
## GRAPH THE OUTPUT NET
graph(ymin=-0.5, ymax=1.5)
out = gcurve(color=color.red, label="NET 3 (reciver)")
for i in range(len(t_arr)): 
    out.plot(t_arr[i], net3.data[i])


<IPython.core.display.Javascript object>


## NOR TESTS


In [7]:
canvas()
## TIME ARRAY FOR PLOTTING ##
t_arr = []

## NETS AND MODULES ## 
## DEFAULT VALUE FOR NETS IS "X"
net1 = net(val="0", id='1') ## driver
net2 = net(val="0", id='2') ## driver
net3 = net(id='3') ## reciever 

nor_gate = module( input=[net1, net2],
                    output=net3,
                    func="NOR",
                    id='1'
                    )

## UPDATE THE MODULES ## 
## DELAY = 5NS 
## X (undefined) = 0.5 
## A CHANGE SHOULD HAPPEN AFTER THE 5 TIME CYCLE 
for i in range(10):
    nor_gate.update()
    net1.update()
    net2.update()
    net3.update()
    t_arr.append(nor_gate.time)

    
## GRAPH INPUT NET 1
graph(ymin=-0.5, ymax=1.5)
n1 = gcurve(color=color.black, label="NET 1 (driver)")
for i in range(len(t_arr)):
    n1.plot(t_arr[i], net1.data[i])

## GRAPH INPUT NET 2
graph(ymin=-0.5, ymax=1.5)
n2 = gcurve(color=color.blue, label="NET 2 (driver)")
for i in range(len(t_arr)):
    n2.plot(t_arr[i], net2.data[i])
    
## GRAPH THE OUTPUT NET
graph(ymin=-0.5, ymax=1.5)
out = gcurve(color=color.red, label="NET 3 (reciver)")
for i in range(len(t_arr)): 
    out.plot(t_arr[i], net3.data[i])

<IPython.core.display.Javascript object>


## EVENT TESTING


In [9]:
scene = canvas() 

## drivers 
net1 = net(id='1') ## use this as a "clock"
net2 = net(id='2')

# receiver 
net3 = net(id='3')

gate = module(input=[net1, net1],
             output=net3,
             func="NAND", ## CHANGE TO NAND OR NOR FOR DIFFERENT FUNCTION 
             id='1')

net1.receivers.append(gate)
net2.receivers.append(gate)

net3.driver = gate
sys = event(nets=[net1, net2, net3])

## set the drivers
while sys.time <= 200:
    clock(sys.time%10, net1)
    clock(sys.time%20, net2)    
    sys.listen()
    gate.update()
    
# print(len(net1.data))
## GRAPH INPUT NET 1
graph(ymin=-0.05, ymax=1.05)
n1 = gcurve(color=color.blue, label="NET 1 (driver)")
for i in range(len(sys.data)):
    n1.plot(sys.data[i], net1.data[i])

## GRAPH INPUT NET 2
graph(ymin=-0.05, ymax=1.05)
n2 = gcurve(color=color.black, label="NET 2 (driver)")
for i in range(len(sys.data)):
    n2.plot(sys.data[i], net2.data[i])
    
## GRAPH THE OUTPUT NET
graph(ymin=-0.05, ymax=1.05)
out = gcurve(color=color.red, label="NET 3 (reciver)")
for i in range(len(sys.data)): 
    out.plot(sys.data[i], net3.data[i])

<IPython.core.display.Javascript object>

# TEST CIRCUIT 

In [4]:
scene = canvas() 

## NETS ##
net1 = net(id='1')
net2 = net(id='2')
net3 = net(id='3')
net4 = net(id='4')
net5 = net(id='5')
net6 = net(id='6') ## OUTPUT 

## GATES ##
gate1 = module(input=[net1, net2],
             output=net4,
             func="NAND", ## CHANGE TO NAND OR NOR FOR DIFFERENT FUNCTION 
             id='1')
gate2 = module(input=[net3, net3], ## not
             output=net5,
             func="NAND", ## CHANGE TO NAND OR NOR FOR DIFFERENT FUNCTION 
             id='2')
gate3 = module(input=[net4, net5],
             output=net6,
             func="NAND", ## CHANGE TO NAND OR NOR FOR DIFFERENT FUNCTION 
             id='3')

## BUILDING THE CIRCUIT ## 
net1.receivers.append(gate1)
net2.receivers.append(gate1)
net3.receivers.append(gate2) ## twice? 
net4.receivers.append(gate3)
net5.receivers.append(gate3)

nets = [net1, net2, net3, net4, net5, net6]
sys = event(nets=nets)

## MANUALLY SET SOME VALUES ## 
net2.value = '1'
net3.value = '0'

## RUN THE CIRCUIT ## 
while sys.time <= 100:
    sys.listen()
    clock(sys.time%10, net1) ## drives a clock 
    clock(sys.time%20, net2)    
    gate1.update()
    gate2.update()
    gate3.update()
    
    
## GRAPH INPUT NET 1
graph(ymin=-0.05, ymax=1.05)
n1 = gcurve(color=color.blue, label="NET 1 (driver)")
for i in range(len(sys.data)):
    n1.plot(sys.data[i], net1.data[i])

## GRAPH INPUT NET 2
graph(ymin=-0.05, ymax=1.05)
n2 = gcurve(color=color.black, label="NET 2 (driver)")
for i in range(len(sys.data)):
    n2.plot(sys.data[i], net2.data[i])
    
## GRAPH THE INPUT NET3
graph(ymin=-0.05, ymax=1.05)
n3 = gcurve(color=color.red, label="NET 3 (driver)")
for i in range(len(sys.data)): 
    n3.plot(sys.data[i], net3.data[i])
    
## GRAPH INPUT NET 4
graph(ymin=-0.05, ymax=1.05)
n4 = gcurve(color=color.blue, label="NET 4 (intermediate)")
for i in range(len(sys.data)):
    n4.plot(sys.data[i], net4.data[i])

## GRAPH INPUT NET 5
graph(ymin=-0.05, ymax=1.05)
n5 = gcurve(color=color.black, label="NET 5 (intermediate)")
for i in range(len(sys.data)):
    n5.plot(sys.data[i], net5.data[i])
    
## GRAPH THE OUTPUT NET 6
graph(ymin=-0.05, ymax=1.05)
out = gcurve(color=color.red, label="NET 6 (output)")
for i in range(len(sys.data)): 
    out.plot(sys.data[i], net6.data[i])

<IPython.core.display.Javascript object>

# D-LATCH 

In [6]:
scene = canvas()

## NETS
## DATA LINE AND CLOCK ## 
clk = net(val="0", id='1')
data = net(val="0", id='2') 
not_data = net(val="0", id='7')

## INTERMEDIATE ##
net3 = net(val="0", id='3')
net4 = net(val="0", id='4')

## OUTPUTS ## 
Q = net(val="0", id='5')
Qnot = net(val="0", id='6')


## GATES 
not_gate = module( input=[data, data],
                output=not_data, 
                func="NAND", 
                id='5')

gate1 = module( input=[data, clk],
                  output=net3,
                  func="NAND",
                  id='1')

gate2 = module( input=[clk, not_data],
                  output=net4,
                  func="NAND",
                  id='2')

gate3 = module( input=[net3, Qnot],
                  output=Q,
                  func="NAND",
                  id='3')

gate4 = module( input=[Q, net4],
                  output=Qnot,
                  func="NAND",
                  id='4')

## SETUP RECIEVERS ## 
not_data.receivers = [gate2]
data.receivers = [gate1, not_gate]
clk.receivers = [gate1, gate2]
net3.receivers = [gate3]
net4.receivers = [gate4]
Q.receivers = [gate4]
Qnot.receivers = [gate3]

## CONSOLIDATING ALL GATES AND NETS ## 
gates = [not_gate, gate1, gate2, gate3, gate4]
nets = [clk, data, not_data, net3, net4, Q, Qnot]
sys = event(nets=nets)

## RUN CIRCUIT FOR 400 TIME UNITS ## 
while sys.time <= 400: 
    sys.listen()
    clock(sys.time%20, clk)
    if sys.time == 200: 
        data.value = "1"
    for gate in gates:
        gate.update()
    
## GRAPHING ## 
## GRAPH INPUT CLK
graph(ymin=-0.05, ymax=1.05)
n1 = gcurve(color=color.blue, label="CLOCK (driver)")
for i in range(len(sys.data)):
    n1.plot(sys.data[i], clk.data[i])

## GRAPH INPUT DATA
graph(ymin=-0.05, ymax=1.05)
n2 = gcurve(color=color.black, label="DATA (driver)")
for i in range(len(sys.data)):
    n2.plot(sys.data[i], data.data[i])
    
## GRAPH THE OUTPUT Q
graph(ymin=-0.05, ymax=1.05)
n5 = gcurve(color=color.red, label="Q (output)")
for i in range(len(sys.data)): 
    n5.plot(sys.data[i], Q.data[i])

## GRAPH THE OUTPUT ~Q
graph(ymin=-0.05, ymax=1.05)
n6 = gcurve(color=color.red, label="~Q (output)")
for i in range(len(sys.data)): 
    n6.plot(sys.data[i], Qnot.data[i])
    

## GRAPH INTERMEDIATE ## 
graph(ymin=-0.05, ymax=1.05)
n3 = gcurve(color=color.green, label="net3 (intermediate)")
for i in range(len(sys.data)): 
    n3.plot(sys.data[i], net3.data[i])

graph(ymin=-0.05, ymax=1.05)
n4 = gcurve(color=color.green, label="net4 (intermediate)")
for i in range(len(sys.data)): 
    n4.plot(sys.data[i], net4.data[i])


<IPython.core.display.Javascript object>

### For the D-LATCH simulation - I took advantage of the "pass-by-reference" aspect of python to signal events into each gate. The random preamble at the start of the "Q" and "~Q" graph is from the initial states of all the nets (active low). The delay is observable from both the "data" graph and the "Q" graph as data goes from low to high . 